# Predictive Modelling: The Census Income UCI Dataset
In this notebook we run through a basic prediction task using the [Census Income UCI Dataset](http://archive.ics.uci.edu/ml/datasets/Census+Income). The task is to predict whether income exceeds $50k/year based on census data.

### Dataset information

This dataset consists of 48842 instances and 15 features. The features take on a mix of categorical/numerical values:

1. **age**: 16+ (continuous).
2. **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
3. **fnlwgt**: Final weight, see below (continuous).
4. **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
5. **education_num**: Total number of years of education (continuous).
6. **marital_status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
7. **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
8. **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
9. **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
10. **sex**: Female, Male.
11. **capital_gain**: continuous.
12. **capital_loss**: continuous.
13. **hours_per_week**: Number of hours worked per week (continuous).
14. **native_country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
15. **Income**: >50k, <=50k. 

The dataset is already split into train-test sets of sizes (2/3, 1/3). Both sets contain missing values, denoted by '?'.

#### Description of fnlwgt (final weight):
The UCI repository lists this additional information for the fnlwgt column:

"The weights on the Current Population Survey (CPS) files are controlled to independent estimates of the civilian noninstitutional population of the US. These are prepared monthly for us by Population Division here at the Census Bureau. We use 3 sets of controls. These are:

1. A single cell estimate of the population 16+ for each state.
2. Controls for Hispanic Origin by age and sex.
3. Controls by Race, age and sex."

### Dependencies
* Python 3+
* Pandas, numpy, scikit-learn

In [1]:
# Standard libraries
import pandas as pd
import numpy as np

# Train and evaluate models
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

## 1. Data cleaning and preprocessing
To be consistent in cleaning our data, we will merge the train and test sets into a full dataset and separate it later for model validation.

In [2]:
# Load data
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
           'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
           'income']

data_train = pd.read_csv("./data/adult.data", sep=', ', header=None, names=columns, engine='python')
data_test = pd.read_csv("./data/adult.test", sep=', ', header=None, skiprows=1, names=columns, engine='python')

display(data_train.head(3))
display(data_test.head(3))

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.


In [3]:
# Merge datasets
data = pd.concat([data_train, data_test])
data

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


### 1.1. Relevant features
The value of the fnlwgt (final weight) column should have no predictive power, at least without further processing. We will drop this column. 

In [4]:
# Drop fnlwgt column
data.drop('fnlwgt', axis=1, inplace=True)

### 1.2. Missing values
Missing values are represented as the string '?'. We first replace them with NaNs.

In [5]:
# Replace '?' with NaN
data.replace('?', np.nan, inplace=True)

In [6]:
# Count missing values
data.isna().sum()

age                  0
workclass         2799
education            0
education_num        0
marital_status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     857
income               0
dtype: int64

The missing values are all within categorical variables. Since we can't usefully infer them, we drop each row with a missing value.

In [7]:
data.dropna(axis=0, inplace=True)

Finally, reset index for convenience and store full dataset.

In [8]:
# reset index
data.reset_index(drop=True, inplace=True)

In [9]:
# Store full dataset
data.to_csv("./data/adult.csv", index=False)

### 1.3. Encode target feature as integers
The target feature (income column) consists of strings of the form below:

In [10]:
data.income

0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
45217    <=50K.
45218    <=50K.
45219    <=50K.
45220    <=50K.
45221     >50K.
Name: income, Length: 45222, dtype: object

We encode income below 50k as 0, and above 50k as 1.

In [11]:
data.income = data.income.str.strip(".")

In [12]:
data.income = data.income.map({"<=50K": 0, ">50K": 1})

In [13]:
data.income.value_counts()

0    34014
1    11208
Name: income, dtype: int64

In [14]:
data

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,33,Private,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0
45218,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,0
45219,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,0
45220,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,0


### 1.4. One-Hot encode the categorical values

In [15]:
data = pd.get_dummies(data)
data.shape

(45222, 104)

In [16]:
data.head(3)

,age,education_num,capital_gain,capital_loss,hours_per_week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,39,13,2174,0,40,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,13,0,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,9,0,0,40,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


### 1.5. Feature scaling

In [17]:
X, y = data.drop('income', axis=1), data.income

In [18]:
scaler = StandardScaler()

In [19]:
scaler.fit(X.loc[:, 'age': 'hours_per_week'])
X.loc[:, 'age': 'hours_per_week'] = scaler.transform(X.loc[:, 'age': 'hours_per_week'])

In [20]:
X.head(5)

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,0.034201,1.128753,0.142888,-0.21878,-0.078120,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.866417,1.128753,-0.146733,-0.21878,-2.326738,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,-0.041455,-0.438122,-0.146733,-0.21878,-0.078120,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1.093385,-1.221559,-0.146733,-0.21878,-0.078120,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-0.798015,1.128753,-0.146733,-0.21878,-0.078120,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2. Data modelling

### 2.1. Train test split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### 2.1. Gradient Boosted Trees
We will use gradient boosted trees, with parameters tuned via grid search cross-validation. First, let's look at the cross-validation scores with default values to get a baseline.

In [22]:
# Gradient boosted trees with default values
gbt = GradientBoostingClassifier(random_state=0)
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(gbt, X, y, cv=kfold)
print("Cross-validation scores: mean {:.3f}, std {:.3f}".format(scores.mean(), scores.std()))

Cross-validation scores: mean 0.863, std 0.001


We'll try tuning the following parameters:
* max_depth (defaults to 3)
* n_estimators (defaults to 100)

We also keep random_state=0 as above for reproducibility.

In [23]:
# Parameter grid to optimise over
param_grid = {
    'random_state': [0],
    'max_depth': [2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200, 250, 300],
}

# Train our model
grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get scores and model information
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Best parameters: {'max_depth': 5, 'n_estimators': 200, 'random_state': 0}
Best cross-validation score: 0.869
Best estimator:
GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


With some tuning of parameters, we see that our accuracy should raise to ~0.869 using max_depth = 5 and n_estimators=200. Let's try our newly fitted model on the test set.

In [24]:
# Get train and test set scores
print("Train set score: {:.3f}".format(grid_search.score(X_train, y_train)))
print("Test set score: {:.3f}".format(grid_search.score(X_test, y_test)))

Train set score: 0.884
Test set score: 0.870


Finally, we can inspect the result of the grid search over different parameters for further tuning.

In [25]:
results = pd.DataFrame(grid_search.cv_results_)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.160780,0.037878,0.007860,0.000821,2,50,0,"{'max_depth': 2, 'n_estimators': 50, 'random_s...",0.847583,0.855079,0.854931,0.850361,0.852573,0.852105,0.002848,24
1,2.187243,0.120700,0.009864,0.000370,2,100,0,"{'max_depth': 2, 'n_estimators': 100, 'random_...",0.851562,0.861566,0.857880,0.853457,0.857585,0.856410,0.003531,23
2,3.099185,0.006702,0.011799,0.000122,2,150,0,"{'max_depth': 2, 'n_estimators': 150, 'random_...",0.855248,0.863777,0.859502,0.856848,0.858322,0.858739,0.002895,21
3,4.104170,0.012035,0.013877,0.000131,2,200,0,"{'max_depth': 2, 'n_estimators': 200, 'random_...",0.858343,0.867168,0.860386,0.859502,0.860829,0.861246,0.003081,20
4,5.089706,0.007495,0.015999,0.000137,2,250,0,"{'max_depth': 2, 'n_estimators': 250, 'random_...",0.859080,0.868790,0.862893,0.860091,0.861566,0.862484,0.003408,17
